In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from textwrap import wrap
import folium
import plotly.graph_objects as go
%matplotlib inline
import os
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [60]:
# Methods
def unpack_column(dataFrame,col_name):
    newDf = dataFrame.from_records(dataFrame[col_name].dropna().tolist())
    return newDf

def unpack_and_destroy(dataFrame,column):
    temp = unpack_column(dataFrame,column)
    for col in temp.columns:
        dataFrame[col] = temp[col]
    #dataFrame = dataFrame.drop(columns=[column])
    return dataFrame

def sort_db_value(db,value,asc=False):
    return db.sort_values(by = value, ascending = asc)

def freq_array(freq_list):
    freq = []
    for i in freq_list:
        freq.append(i)
    return freq

def set_labels(freq, ax,horz_shift=.2,vert_shift=1000,w="bold",fsize=10,c="b"):
    for i, v in enumerate(freq):
        ax.text(i-horz_shift,v+vert_shift, str(round(v,2)), weight = w,fontsize=fsize,color = c)
        
def wrap_axis_labels(labels_list,wrap_length=13):
    labels_list = [ '\n'.join(wrap(l, wrap_length)) for l in labels_list]
    return labels_list

In [61]:
donations_df = pd.read_csv('Data/Donations_File.csv')
user_df = pd.read_csv('Data/User_Donations_File_Acct.csv')

In [62]:
user_df.head()

,User ID,Gender,Birthday,Date created
0,1078,N,NaN,2018-09-27 20:28:13.579708-04:00
1,2555,N,NaN,2019-06-16 13:06:50.910577-04:00
2,2792,N,NaN,2019-08-13 11:15:30.053323-04:00
3,142,N,NaN,2017-12-12 19:00:00-05:00
4,369,M,6/25/88,2018-03-20 06:38:25.606480-04:00


In [63]:
donations_df.head(5)

,Donation ID,User ID,Amount,Date,To Charity,To Charity EIN,Advised Charity,Advised Charity EIN,Message to Charity2,Message to Charity1,Unnamed: 10,In Honor Of?,Event?,Cover fee?,Tip,Recurring Status,Givz Everywehre?
0,2753,2837.0,25.0,2019-09-11 03:17:52.392953-04:00,Social Good Fund,461323531,Room to Grow National Inc,134012096,NaN,NaN,False,False,NaN,False,0.0,O,NaN
1,2752,1629.0,20.0,2019-09-10 22:55:52.758423-04:00,Social Good Fund,461323531,Wildlife Conservation Society,131740011,NaN,NaN,False,False,NaN,False,0.0,M,NaN
2,2751,1134.0,21.0,2019-09-10 22:55:49.962729-04:00,Social Good Fund,461323531,Benchmark School,231728895,NaN,NaN,False,False,NaN,False,0.0,M,NaN
3,2750,2566.0,140.0,2019-09-10 22:29:05.231148-04:00,Social Good Fund,461323531,Play Soccer to Give Corp,455071137,NaN,NaN,False,False,NaN,False,0.0,O,k4lfall19-1
4,2749,2847.0,1.0,2019-09-10 16:52:30.401878-04:00,Social Good Fund,461323531,Room to Grow National Inc,134012096,NaN,NaN,False,False,NaN,False,0.0,O,NaN


In [64]:
donations_df.shape

(2536, 17)

In [65]:
Num_donations_per_user_df = donations_df.groupby("User ID").size().reset_index(name="Count").sort_values(by="Count",ascending=False)

In [66]:
Num_donations_per_user_df.head()

,User ID,Count
60,169.0,185
18,48.0,96
65,326.0,80
47,139.0,54
68,364.0,42


In [67]:
Total_donations_per_user_df = donations_df.groupby("User ID")['Amount','Tip'].sum().reset_index().sort_values(by="Tip",ascending=False)

In [68]:
Total_donations_per_user_df.head()

,User ID,Amount,Tip
894,2094.0,1802.5,180.25
935,2156.0,1649.0,77.50
105,463.0,2860.0,62.50
942,2166.0,649.0,55.00
557,1295.0,495.0,47.50


In [69]:
df_merge_col = pd.merge(Num_donations_per_user_df, Total_donations_per_user_df, on='User ID')

In [70]:
df_merge_col.head()

,User ID,Count,Amount,Tip
0,169.0,185,822.19,1.90
1,48.0,96,528.50,3.20
2,326.0,80,4350.00,13.75
3,139.0,54,269.60,2.50
4,364.0,42,470.00,15.50


In [71]:
df_merge_col["AVG_Donation"] = round((df_merge_col["Amount"] / df_merge_col["Count"]),2)

In [72]:
df_merge_col["AVG_Tip"] = round((df_merge_col["Tip"] / df_merge_col["Count"]),2)

In [73]:
df_merge_col.sort_values(by="AVG_Tip",ascending=False).head()

,User ID,Count,Amount,Tip,AVG_Donation,AVG_Tip
316,2094.0,1,1802.5,180.25,1802.5,180.25
206,2156.0,2,1649.0,77.50,824.5,38.75
202,2166.0,2,649.0,55.00,324.5,27.50
66,463.0,4,2860.0,62.50,715.0,15.62
502,2564.0,1,145.0,14.50,145.0,14.50


In [74]:
df_merge_col["Tip"].sum()

1326.27

In [76]:
df_merge_col.head()

,User ID,Count,Amount,Tip,AVG_Donation,AVG_Tip
0,169.0,185,822.19,1.90,4.44,0.01
1,48.0,96,528.50,3.20,5.51,0.03
2,326.0,80,4350.00,13.75,54.38,0.17
3,139.0,54,269.60,2.50,4.99,0.05
4,364.0,42,470.00,15.50,11.19,0.37
